In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

# Tester results.


In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
data = pd.read_csv("test_result.txt", sep=' ', header=None, names=["lib", 'test', 'test_name'] + [i for i in range(10)])

In [ ]:
values = [i for i in range(10)]
data["mean"] = data[values].mean(axis=1)
data["std"]  = data[values].std(axis=1)
data["testid"] = data["test"] + "_" + data["test_name"]
tests = set(data['testid'].tolist())
libs = set(data['lib'].tolist())

## Passed and Failed tests

In [ ]:
passedf = pd.DataFrame(index=libs, columns=tests).fillna(4.75)

dataslice = data[['lib', 'testid']]
for index, row in dataslice.iterrows():
    passedf.set_value(row['lib'], row['testid'], 2.2)

In [ ]:
sns.set(font_scale=2)
sns.set_style("whitegrid")
row_labels = libs
col_labels = tests
plt.figure(figsize=(20,10))
plt.imshow(passedf, interpolation='none', vmin=0, vmax=5, cmap="spectral")
plt.xticks(range(len(tests)), col_labels, rotation=90)
plt.yticks(range(len(libs)), row_labels)
plt.show()

## Results by each test.
There are only libs that passed the test.

In [ ]:
# sns.set_style("whitegrid")
sns.set(font_scale=5)
for test in tests: 
    plt.figure(figsize=(45,10))
    ax = sns.barplot(x="lib", y="mean", data=data[data["testid"] == test])
    sns.plt.title(test, fontsize=100)
    ax.set(xlabel='', ylabel='mean time')
    plt.show()

## Raw results 
with mean and std

In [ ]:
pd.set_option('display.max_rows', len(data))
data

#### Made by dokhlopkov (ohld@github, ohl.d@yandex.ru)